In [1]:
from datetime import datetime

from polymarket_api import poly_client


client = poly_client

In [33]:
from datetime import timedelta


ts = client.get_timeseries(
    "82927494066775506689099766141381125044040984047946118898947348013267483041764",
    datetime.now() - timedelta(days=1),
    datetime.now(),
    60,
)
ts

https://clob.polymarket.com/prices-history?market=82927494066775506689099766141381125044040984047946118898947348013267483041764&startTs=1741104731&endTs=1741191131&fidelity=60


{'history': [{'t': 1741107605, 'p': 0.265},
  {'t': 1741111205, 'p': 0.285},
  {'t': 1741114805, 'p': 0.3},
  {'t': 1741118405, 'p': 0.33},
  {'t': 1741122005, 'p': 0.335},
  {'t': 1741125605, 'p': 0.345},
  {'t': 1741129205, 'p': 0.345},
  {'t': 1741132804, 'p': 0.345},
  {'t': 1741136404, 'p': 0.335},
  {'t': 1741140005, 'p': 0.34},
  {'t': 1741143605, 'p': 0.335},
  {'t': 1741147205, 'p': 0.33},
  {'t': 1741150805, 'p': 0.335},
  {'t': 1741154405, 'p': 0.34},
  {'t': 1741158005, 'p': 0.345},
  {'t': 1741161605, 'p': 0.345},
  {'t': 1741165205, 'p': 0.345},
  {'t': 1741168804, 'p': 0.35},
  {'t': 1741172405, 'p': 0.39},
  {'t': 1741176004, 'p': 0.39},
  {'t': 1741179604, 'p': 0.42},
  {'t': 1741183204, 'p': 0.385},
  {'t': 1741186804, 'p': 0.375},
  {'t': 1741190405, 'p': 0.365},
  {'t': 1741191124, 'p': 0.36}]}

In [4]:
ss_markets = client.get_simplified_markets()
ss_markets["data"][0]

{'condition_id': '0x9deb0baac40648821f96f01339229a422e2f5c877de55dc4dbf981f95a1e709c',
 'rewards': {'rates': None, 'min_size': 0, 'max_spread': 0},
 'tokens': [{'token_id': '14270523446080509320829200481895961480205553513304203367521919818541658424782',
   'outcome': 'Chiefs',
   'price': 1,
   'winner': False},
  {'token_id': '93110170397161149119544349457822484949376809039410140245101963942162463626903',
   'outcome': 'Raiders',
   'price': 0,
   'winner': False}],
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False}

In [2]:
import time


def find_crypto_markets(limit=10, max_queries=3, cursor="MA==", condition=lambda m: m["tags"] and "Crypto" in m["tags"]):
    found_any = False
    crypto_markets = []

    queries_made = 0
    while (queries_made < max_queries) and (len(crypto_markets) < limit):
        print(f"Fetching markets with cursor {cursor}")

        markets = client.get_markets(next_cursor=cursor)
        cursor = markets["next_cursor"]

        print(f"Loaded {len(markets["data"])} markets")

        data = [m for m in markets["data"] if condition(m)]
        if data:
            if not found_any:
                print(f"First cursor with data found: {cursor}")
                found_any = True

        print(f"Found {len(data)} crypto markets")

        crypto_markets += data
        queries_made += 1

        time.sleep(0.5)

    markets = client.get_markets()

    if not crypto_markets:
        print("No crypto markets found")
    else:
        print(f"Crypto markets found ({len(crypto_markets)})")
    
    print(f"Next cursor: {cursor}")

    return crypto_markets


def is_btc_related_market(m):
    return "market_slug" in m.keys() and ("btc" in m["market_slug"])

# def is_

markets = find_crypto_markets(limit=10, cursor="OTAwMA==", condition=is_btc_related_market)

Fetching markets with cursor OTAwMA==
Loaded 500 markets
First cursor with data found: OTUwMA==
Found 3 crypto markets
Fetching markets with cursor OTUwMA==
Loaded 500 markets
Found 2 crypto markets
Fetching markets with cursor MTAwMDA=
Loaded 500 markets
Found 1 crypto markets
Crypto markets found (6)
Next cursor: MTA1MDA=


In [3]:
markets[0]

{'enable_order_book': False,
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False,
 'accepting_order_timestamp': '2024-07-01T18:48:13Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.001,
 'condition_id': '0x7c1a2610911ea83b5e9a44d89201635c69919324d45e8deb92ef422b1dddc12b',
 'question_id': '0x3cc03c5ebbdcb5ba3b334e5913d49028e39b8fb6bb90c6921a82be5a15673d8b',
 'question': 'Will $BTC reach 70k in July?',
 'description': 'This market will immediately resolve to "Yes" if any Binance 1 minute candle for Bitcoin (BTCUSDT) between July 1, 2024, 00:00 and July 31, 2024, 23:59 in the ET timezone has a final "High" price of $70,000.00 or higher. Otherwise, this market will resolve to "No."\n\nThe resolution source for this market is Binance, specifically the BTCUSDT "High" prices available at https://www.binance.com/en/trade/BTC_USDT, with the chart settings on "1m" for one-minute candles selected on the top bar.\n\nPlease note that the outcome of this market depe

In [18]:
[("https://polymarket.com/event/" + m["market_slug"], m["condition_id"]) for m in markets]


[('https://polymarket.com/event/will-btc-reach-70k-in-july',
  '0x7c1a2610911ea83b5e9a44d89201635c69919324d45e8deb92ef422b1dddc12b'),
 ('https://polymarket.com/event/will-btc-reach-80k-in-july',
  '0x10bfd2053bfe798aa0fec72fc4aaf465b17b60afb453919ad2f366770042a8d7'),
 ('https://polymarket.com/event/will-btc-reach-75k-in-july',
  '0x409bbbd78d6bf45cd67048fd9ef80f473152f82aef9f3dd275a616af7474cab7'),
 ('https://polymarket.com/event/will-btc-hit-50k-or-70k-first',
  '0xf2830173b5e239e7db0509d4489105107225101cce05b41c1818b4b78e00d6e8'),
 ('https://polymarket.com/event/will-btc-or-eth-reach-all-time-high-first-2024',
  ''),
 ('https://polymarket.com/event/will-a-presidential-candidate-say-mog-at-btc-conference',
  '0x69e439c06b8ce7575d2dcb209a79a362981ec2c949cee41123e5c248807b9ed1')]

In [ ]:
market = client.get_market("0xa9444a9067fb2958ac36dd3f0ea25c07da5c39dc595643280b5ec037334b75de")
market

dict_keys(['enable_order_book', 'active', 'closed', 'archived', 'accepting_orders', 'accepting_order_timestamp', 'minimum_order_size', 'minimum_tick_size', 'condition_id', 'question_id', 'question', 'description', 'market_slug', 'end_date_iso', 'game_start_time', 'seconds_delay', 'fpmm', 'maker_base_fee', 'taker_base_fee', 'notifications_enabled', 'neg_risk', 'neg_risk_market_id', 'neg_risk_request_id', 'icon', 'image', 'rewards', 'is_50_50_outcome', 'tokens', 'tags'])

In [5]:
markets[3]

{'enable_order_book': False,
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False,
 'accepting_order_timestamp': '2024-07-12T17:34:12Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.001,
 'condition_id': '0xf2830173b5e239e7db0509d4489105107225101cce05b41c1818b4b78e00d6e8',
 'question_id': '0x53de58d6afe1f85a7cce005214214b10c5c171ac99be33d0c5f58cd8ec4694eb',
 'question': 'Will $BTC hit $50k or $70k first?',
 'description': 'This is a market on whether Bitcoin ($BTC) will first reach $50,000.00 USD or $70,000.00 USD between July 12, 2024, 12 PM ET, and December 31, 2024, 11:59 PM ET.\n\nThis market will resolve to "50k" if $BTC drops to $50,000.00 USD or less first within the market timeframe. This market will resolve to "70k" if $BTC reaches $70,000.00 or more first within the market timeframe.\n\nIf the price of $BTC neither drops to $50,000.00 or below nor reaches $70,000.00 or more within the market timeframe, this market will resolve 50-50.\n\nThe re

In [2]:
from polymarket_api import get_poly_timeseries


condition_id = "0xf2830173b5e239e7db0509d4489105107225101cce05b41c1818b4b78e00d6e8"

df = get_poly_timeseries(condition_id, 720)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 34 entries, 2024-07-13 02:00:03 to 2024-07-29 14:00:02
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   50k     34 non-null     float64
 1   70k     34 non-null     float64
dtypes: float64(2)
memory usage: 816.0 bytes


In [3]:
df.head()

,50k,70k
date,,
2024-07-13 02:00:03,0.580,0.420
2024-07-13 14:00:03,0.575,0.425
2024-07-14 02:00:02,0.530,0.470
2024-07-14 14:00:03,0.480,0.520
2024-07-15 02:00:03,0.465,0.535


In [31]:
end_ts = datetime.fromisoformat(markets[3]["end_date_iso"])

ts = client.get_timeseries(
    "82927494066775506689099766141381125044040984047946118898947348013267483041764",
    end_ts - timedelta(days=2),
    end_ts,
    1,
)
ts

https://clob.polymarket.com/prices-history?market=82927494066775506689099766141381125044040984047946118898947348013267483041764&startTs=1735430400&endTs=1735603200&fidelity=1


{'history': [{'t': 1741104545, 'p': 0.235},
  {'t': 1741104605, 'p': 0.235},
  {'t': 1741104664, 'p': 0.235},
  {'t': 1741104724, 'p': 0.235},
  {'t': 1741104784, 'p': 0.235},
  {'t': 1741104845, 'p': 0.24},
  {'t': 1741104904, 'p': 0.245},
  {'t': 1741104965, 'p': 0.245},
  {'t': 1741105025, 'p': 0.245},
  {'t': 1741105084, 'p': 0.255},
  {'t': 1741105144, 'p': 0.235},
  {'t': 1741105205, 'p': 0.235},
  {'t': 1741105264, 'p': 0.235},
  {'t': 1741105325, 'p': 0.235},
  {'t': 1741105385, 'p': 0.235},
  {'t': 1741105445, 'p': 0.235},
  {'t': 1741105505, 'p': 0.235},
  {'t': 1741105564, 'p': 0.235},
  {'t': 1741105625, 'p': 0.235},
  {'t': 1741105685, 'p': 0.235},
  {'t': 1741105744, 'p': 0.235},
  {'t': 1741105804, 'p': 0.235},
  {'t': 1741105864, 'p': 0.23},
  {'t': 1741105924, 'p': 0.235},
  {'t': 1741105985, 'p': 0.235},
  {'t': 1741106045, 'p': 0.235},
  {'t': 1741106105, 'p': 0.25},
  {'t': 1741106165, 'p': 0.235},
  {'t': 1741106225, 'p': 0.25},
  {'t': 1741106285, 'p': 0.235},
  {